<a href="https://colab.research.google.com/github/jasonheesanglee/LLM_Study/blob/main/Creating_RAG_with_OpenSource_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -qqq python-multipart openai cohere kaleido langchain tiktoken sentence_transformers transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -qqq bitsandbytes
!pip install -qqq -U git+https://github.com/huggingface/transformers.git
!pip install -qqq -U git+https://github.com/huggingface/peft.git
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install transformers[torch] torch

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
model_name = 'kyujinpy/Ko-PlatYi-6B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map='auto')

tokenizer_config.json:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (n

In [9]:
device = 'cuda:0'
messages = [
    {'role':'user', 'content':'은행의 기준 금리에 대해서 설명해줘'}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors='pt')

model_input = encodeds.to(device)

generated_ids = model.generate(model_input, max_new_tokens=10000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|startoftext|> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

은행의 기준 금리에 대해서 설명해줘 [/INST]
대출이자. 이자율이 어떻게 되나요? [/INST]<|endoftext|>


In [11]:
import locale
def getpreferredencoding(do_setlocale=True):
    return 'UTF-8'

locale.getpreferredencoding = getpreferredencoding

In [12]:
!pip -qqq install langchain pypdf chromadb sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.2 MB/s eta 0:00:00


In [22]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

In [24]:
prompt_template = '''
### [INST]
Instruction: Answer the question based on your bank economy knowledge.
Here is context to help:

{context}

### Question
{question}
[/INST]
'''

koplatyllm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)
llm_chain = LLMChain(llm=koplatyllm, prompt=prompt)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/LangChain/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_id = 'jhgan/ko-sbert-nli'
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_id,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
    search_type='similarity',
    search_kwargs={'k':3}
)


In [27]:
rag_chain = (
    {'context':retriever, 'question':RunnablePassthrough()}
    | llm_chain
)

In [28]:
import warnings
warnings.filterwarnings('ignore')


In [33]:
result = rag_chain.invoke('혁신성장 정책 금융에서 인공지능이 중요한가? 근거와 함께 논리적으로 설명해줘')

for i in result['context']:
    print(f"주어진 근거 : {i.page_content} \n출처: {i.metadata['source']} - {i.metadata['page']}\n\n")

print(f"\n답변 : {result['text']}")

주어진 근거 : 혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2 호 | 9 |▶다양한 데이터나 복잡한 자료 속에서 핵심적인 특징을 요약하는 ①데이터 추상화 기술, 방대한 지식
체계를 이용하는 ②빅데이터 기술, 빅데이터를 처리하기 위한 ③고성능 컴퓨팅 기술이 인공지능 
구현의 핵심임
￮데이터를 추상화하는 방법은 크게 인공신경망 (ANN), 심층신경망 (DNN), 합성곱신경망 (CNN) 및 
순환신경망 (RNN) 등으로 구분됨
[인공지능 데이터 추상화 기술]
구분 특징 장점 단점
인공신경망 (ANN)
Artificial Neural 
Network사람의 신경망 원리와 구조를 모방하여 만든 
기계학습 알고리즘으로 , 입력층 , 출력층 , 은닉층
으로 구성모든 비선형 함수 
학습이 가능알고리즘을 최적화
하기 어려운 학습 
환경 발생
심층신경망 (DNN)
Deep Neural 
Network입력층과 출력층 사이에 2개 이상의 은닉층 
출처: /content/drive/MyDrive/Colab Notebooks/LangChain/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf - 8


주어진 근거 : | 8 | CIS이슈리포트 2022-2 호 ▶(주요품목 ② : 인공지능 ) 정보통신 테마 내 기술분야 중 정책금융 공급규모 증가율이 가장 높은 
능동형컴퓨팅 분야의 경우, 인공지능 품목의 정책금융 공급 비중이 가장 높으며 , 이는 빅데이터 
분석기술의 발전으로 인해 인공지능의 활용처가 넓어짐에 따른 것으로 분석됨
[능동형컴퓨팅 분야 내 기술품목별 혁신성장 정책금융 공급액 추이]
(단위: 억 원)
주: 스마트물류시스템 품목은 2021 년부터 신규 품목으로 편임
▶인공지능은 인간의 학습능력과 추론·지각능력 , 자연언어 이해능력 등을 프로그램으로 구현한 기술로 , 
컴퓨터가 인간의 지능적인 행동을 모방하는 방향으로 발전하고 있음
￮인공지능은 사람의 두뇌가 복잡한 연산을 수행하는 점을 모방